## Imports

In [2]:
! pip install mlxtend
! pip install xlrd

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import sklearn as sk
from collections import defaultdict, Counter

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report # type: ignore
from sklearn import tree
from yellowbrick.classifier import ConfusionMatrix # type: ignore

In [5]:
df = pd.read_csv('Texto_Filtragem.csv')
df

,Id,Idade,Faixa idade,Gênero,Cor/raca/etnia,PCD,Vive no Brasil,Estado onde mora,UF onde mora,Região onde Mora,...,Qual sua situação atual de trabalho?,Setor,Numero de Funcionários,Cargo Atual,Nível,Faixa salarial,Quanto tempo de experiência na área de dados você tem?,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?,Existe faculdade no Estado
0,001b2d1qtli8t9z7oqgdhj001b2d4i0g,31,30-34,Masculino,Branca,0,1,Minas Gerais (MG),MG,Sudeste,...,Empregado (CLT),Finanças ou Bancos,Acima de 3.000,Cientista de Dados/Data Scientist,Sênior,de R$ 12.001/mês a R$ 16.000/mês,de 3 a 4 anos,Não tive experiência na área de TI/Engenharia ...,0,1
1,0026aa3fwd78u0026asg7456tfkjg2cs,30,30-34,Masculino,Branca,0,1,Espírito Santo (ES),ES,Sudeste,...,Empreendedor ou Empregado (CNPJ),Finanças ou Bancos,de 101 a 500,Analista de BI/BI Analyst,Pleno,de R$ 6.001/mês a R$ 8.000/mês,de 3 a 4 anos,de 3 a 4 anos,0,0
2,00r21rb9pusd1b0v7ew00r21rw3dy69w,37,35-39,Feminino,Amarela,0,1,São Paulo (SP),SP,Sudeste,...,Empregado (CLT),Finanças ou Bancos,Acima de 3.000,Analista de Dados/Data Analyst,Júnior,de R$ 6.001/mês a R$ 8.000/mês,de 1 a 2 anos,Menos de 1 ano,0,1
3,00v0az4g792svil00vn6y1kfm9hq8vy9,34,30-34,Masculino,Branca,0,1,Minas Gerais (MG),MG,Sudeste,...,Empreendedor ou Empregado (CNPJ),Outra Opção,de 101 a 500,Analista de Dados/Data Analyst,Júnior,de R$ 4.001/mês a R$ 6.000/mês,Menos de 1 ano,Menos de 1 ano,0,1
4,015qqbrh4rr015qje9gvkbsb7ti1n5df,30,30-34,Masculino,Branca,0,1,Rio de Janeiro (RJ),RJ,Sudeste,...,Empreendedor ou Empregado (CNPJ),Tecnologia/Fábrica de Software,de 51 a 100,Engenheiro de Dados/Arquiteto de Dados/Data En...,Pleno,de R$ 8.001/mês a R$ 12.000/mês,de 1 a 2 anos,Não tive experiência na área de TI/Engenharia ...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,zwm2c16e32000c6iaswzwm2c1jv6swo0,36,35-39,Feminino,Parda,0,1,São Paulo (SP),SP,Sudeste,...,Empregado (CLT),Área da Saúde,de 101 a 500,Analista de Dados/Data Analyst,Pleno,de R$ 6.001/mês a R$ 8.000/mês,de 3 a 4 anos,Não tive experiência na área de TI/Engenharia ...,0,1
2903,zws1zie6ro3he9ln337yufzws1zyj7nw,23,22-24,Masculino,Parda,0,1,Goiás (GO),GO,Centro-oeste,...,Empregado (CLT),Varejo,de 51 a 100,Analista de Dados/Data Analyst,Júnior,de R$ 3.001/mês a R$ 4.000/mês,Menos de 1 ano,de 3 a 4 anos,0,0
2904,zxvrsh0xk8z95k76e63zxvrshjgyigq0,27,25-29,Feminino,Parda,0,1,Espírito Santo (ES),ES,Sudeste,...,Empregado (CLT),Varejo,Acima de 3.000,Data Product Manager/ Product Manager (PM/APM/...,Pleno,de R$ 8.001/mês a R$ 12.000/mês,de 3 a 4 anos,Não tive experiência na área de TI/Engenharia ...,0,0
2905,zy7hsey7pigk6d4a4zy7hsm7bh1vcozc,28,25-29,Masculino,Branca,0,1,São Paulo (SP),SP,Sudeste,...,Empregado (CLT),Finanças ou Bancos,de 501 a 1.000,Cientista de Dados/Data Scientist,Júnior,de R$ 8.001/mês a R$ 12.000/mês,de 1 a 2 anos,Não tive experiência na área de TI/Engenharia ...,0,1


In [6]:
df.dtypes

,0
Id,object
Idade,int64
Faixa idade,object
Gênero,object
Cor/raca/etnia,object
PCD,int64
Vive no Brasil,int64
Estado onde mora,object
UF onde mora,object
Região onde Mora,object


## Descrição dos dados

In [7]:
df.describe()

,Idade,PCD,Vive no Brasil,Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?,Existe faculdade no Estado
count,2907.000000,2907.00000,2907.0,2907.0,2907.000000
mean,30.484692,0.02064,1.0,0.0,0.537324
std,6.381809,0.14220,0.0,0.0,0.498691
min,18.000000,0.00000,1.0,0.0,0.000000
25%,26.000000,0.00000,1.0,0.0,0.000000
50%,29.000000,0.00000,1.0,0.0,1.000000
75%,34.000000,0.00000,1.0,0.0,1.000000
max,70.000000,1.00000,1.0,0.0,1.000000


## Remover Atributos

In [12]:
#df.drop(['UF onde mora','Estado onde mora'], axis=1, inplace=True)
df = df[~(df['Vive no Brasil']=='Inválido')]
#df.drop(['Id'], axis=1, inplace=True)
df

,Idade,Faixa idade,Gênero,Cor/raca/etnia,PCD,Vive no Brasil,Região onde Mora,Nível de Ensino,Área de Formação,Qual sua situação atual de trabalho?,Setor,Numero de Funcionários,Cargo Atual,Nível,Faixa salarial,Quanto tempo de experiência na área de dados você tem?,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?,Existe faculdade no Estado
0,31,30-34,Masculino,Branca,0,1,Sudeste,Doutorado ou Phd,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Finanças ou Bancos,Acima de 3.000,Cientista de Dados/Data Scientist,Sênior,de R$ 12.001/mês a R$ 16.000/mês,de 3 a 4 anos,Não tive experiência na área de TI/Engenharia ...,0,1
1,30,30-34,Masculino,Branca,0,1,Sudeste,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Empreendedor ou Empregado (CNPJ),Finanças ou Bancos,de 101 a 500,Analista de BI/BI Analyst,Pleno,de R$ 6.001/mês a R$ 8.000/mês,de 3 a 4 anos,de 3 a 4 anos,0,0
2,37,35-39,Feminino,Amarela,0,1,Sudeste,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Finanças ou Bancos,Acima de 3.000,Analista de Dados/Data Analyst,Júnior,de R$ 6.001/mês a R$ 8.000/mês,de 1 a 2 anos,Menos de 1 ano,0,1
3,34,30-34,Masculino,Branca,0,1,Sudeste,Doutorado ou Phd,Economia/ Administração / Contabilidade / Fina...,Empreendedor ou Empregado (CNPJ),Outra Opção,de 101 a 500,Analista de Dados/Data Analyst,Júnior,de R$ 4.001/mês a R$ 6.000/mês,Menos de 1 ano,Menos de 1 ano,0,1
4,30,30-34,Masculino,Branca,0,1,Sudeste,Estudante de Graduação,Computação / Engenharia de Software / Sistemas...,Empreendedor ou Empregado (CNPJ),Tecnologia/Fábrica de Software,de 51 a 100,Engenheiro de Dados/Arquiteto de Dados/Data En...,Pleno,de R$ 8.001/mês a R$ 12.000/mês,de 1 a 2 anos,Não tive experiência na área de TI/Engenharia ...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,36,35-39,Feminino,Parda,0,1,Sudeste,Pós-graduação,Economia/ Administração / Contabilidade / Fina...,Empregado (CLT),Área da Saúde,de 101 a 500,Analista de Dados/Data Analyst,Pleno,de R$ 6.001/mês a R$ 8.000/mês,de 3 a 4 anos,Não tive experiência na área de TI/Engenharia ...,0,1
2903,23,22-24,Masculino,Parda,0,1,Centro-oeste,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Varejo,de 51 a 100,Analista de Dados/Data Analyst,Júnior,de R$ 3.001/mês a R$ 4.000/mês,Menos de 1 ano,de 3 a 4 anos,0,0
2904,27,25-29,Feminino,Parda,0,1,Sudeste,Graduação/Bacharelado,Outras Engenharias,Empregado (CLT),Varejo,Acima de 3.000,Data Product Manager/ Product Manager (PM/APM/...,Pleno,de R$ 8.001/mês a R$ 12.000/mês,de 3 a 4 anos,Não tive experiência na área de TI/Engenharia ...,0,0
2905,28,25-29,Masculino,Branca,0,1,Sudeste,Graduação/Bacharelado,Química / Física,Empregado (CLT),Finanças ou Bancos,de 501 a 1.000,Cientista de Dados/Data Scientist,Júnior,de R$ 8.001/mês a R$ 12.000/mês,de 1 a 2 anos,Não tive experiência na área de TI/Engenharia ...,0,1


In [13]:
df.dropna(inplace = True)
df.drop(df.loc[(df['Cargo Atual'] == 'Economista')].index, inplace=True)
df.drop(df.loc[(df['Cargo Atual'] == 'Analista de Negócios/Business Analyst')].index, inplace=True)
df.drop(df.loc[(df['Cargo Atual'] == 'Professor/Pesquisador')].index, inplace=True)
df.drop(df.loc[(df['Cargo Atual'] == 'Desenvolvedor/ Engenheiro de Software/ Analista de Sistemas')].index, inplace=True)
df.drop(df.loc[(df['Cargo Atual'] == 'Analytics Engineer')].index, inplace=True)
df.drop(df.loc[(df['Cargo Atual'] == 'Outras Engenharias (não inclui dev)')].index, inplace=True)
df.drop(df.loc[(df['Cargo Atual'] == 'Outra Opção')].index, inplace=True)
df.drop(df.loc[(df['Cargo Atual'] == 'Analista de Suporte/Analista Técnico')].index, inplace=True)
df.drop(df.loc[(df['Faixa salarial'] == 'de R$ 101/mês a R$ 2.000/mês')].index, inplace=True)



In [19]:
#df.drop(['Idade'], axis=1, inplace=True) #Removido pois Faixa de Idade faz o papel alem de ser mt especifico
df.drop(['Vive no Brasil'], axis=1, inplace=True) #Todos vivem no brasil praticamente alem de não querer pessoas que não vivem no brasil
df.drop(['Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?'], axis=1, inplace= True)# poucos dados

## Teste para ver se tira PCD & Raça

In [21]:
print(df['PCD'].value_counts(normalize=True))

PCD
0    0.979353
1    0.020647
Name: proportion, dtype: float64


In [22]:
df.drop(['PCD'], axis=1, inplace=True) #Poucos PCD, pode gerar ruido


In [24]:
print(df['Cor/raca/etnia'].value_counts(normalize=True)) #Distribuição ok

Cor/raca/etnia
Branca                  0.645217
Parda                   0.240881
Preta                   0.073297
Amarela                 0.028906
Prefiro não informar    0.007571
Indígena                0.002409
Outra                   0.001721
Name: proportion, dtype: float64


In [25]:
print(df['Existe faculdade no Estado'].value_counts(normalize=True)) #Distribuição ok

Existe faculdade no Estado
1    0.537164
0    0.462836
Name: proportion, dtype: float64


In [23]:
df.dtypes

,0
Faixa idade,object
Gênero,object
Cor/raca/etnia,object
Região onde Mora,object
Nível de Ensino,object
Área de Formação,object
Qual sua situação atual de trabalho?,object
Setor,object
Numero de Funcionários,object
Cargo Atual,object


## Transformar em binario

In [32]:
df_encoded = pd.get_dummies(df)
df_encoded = df_encoded.astype(bool) #transformado em bool pq o collab tava reclamando
#criar nova df para poder trabalhar com os valores em binario sem alterar a df principal
df_encoded

,Existe faculdade no Estado,Faixa idade_17-21,Faixa idade_22-24,Faixa idade_25-29,Faixa idade_30-34,Faixa idade_35-39,Faixa idade_40-44,Faixa idade_45-49,Faixa idade_50-54,Faixa idade_55+,...,Quanto tempo de experiência na área de dados você tem?_de 4 a 6 anos,Quanto tempo de experiência na área de dados você tem?_de 5 a 6 anos,Quanto tempo de experiência na área de dados você tem?_de 7 a 10 anos,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?_Mais de 10 anos,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?_Menos de 1 ano,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?_Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?_de 1 a 2 anos,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?_de 3 a 4 anos,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?_de 5 a 6 anos,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?_de 7 a 10 anos
0,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2903,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2904,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2905,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


## Geração de conjuntos baseados em repetição

In [31]:
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)
# achar conjuntos que repetem para posteriormente poderem ser utilizados para criar regras
frequent_itemsets
#suppor = porcentagem que o atributo aparece como True
#itemsets = atributo que support se refere



,support,itemsets
0,0.537164,(Existe faculdade no Estado)
1,0.115623,(Faixa idade_22-24)
2,0.373021,(Faixa idade_25-29)
3,0.263937,(Faixa idade_30-34)
4,0.126290,(Faixa idade_35-39)
...,...,...
3262,0.050241,"(Existe faculdade no Estado, Numero de Funcion..."
3263,0.050585,"(Numero de Funcionários_Acima de 3.000, Área d..."
3264,0.066414,"(Numero de Funcionários_Acima de 3.000, Região..."
3265,0.055403,"(Numero de Funcionários_Acima de 3.000, Região..."


## Criação de regras

In [34]:
rules = association_rules(frequent_itemsets,metric="confidence",min_threshold=0.6)
#rules = association_rules    # cria as regras
#   frequent_itemsets,         # a partir dos conjuntos frequentes faça ...
#   metric="confidence",       # usar confiança (% alta)
#    min_threshold=0.6          # manter regras com confiança de 60% ou mais

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Existe faculdade no Estado),(Gênero_Masculino),0.537164,0.758775,0.402959,0.750160,0.988646,1.0,-0.004628,0.965519,-0.024211,0.451252,-0.035713,0.640613
1,(Existe faculdade no Estado),(Cor/raca/etnia_Branca),0.537164,0.645217,0.346524,0.645099,0.999818,1.0,-0.000063,0.999669,-0.000393,0.414574,-0.000331,0.591083
2,(Região onde Mora_Sudeste),(Existe faculdade no Estado),0.609429,0.537164,0.497591,0.816488,1.519996,1.0,0.170228,2.522098,0.875906,0.766702,0.603505,0.871409
3,(Existe faculdade no Estado),(Região onde Mora_Sudeste),0.537164,0.609429,0.497591,0.926329,1.519996,1.0,0.170228,5.301580,0.739147,0.766702,0.811377,0.871409
4,(Existe faculdade no Estado),(Qual sua situação atual de trabalho?_Empregad...,0.537164,0.800413,0.446662,0.831518,1.038862,1.0,0.016709,1.184621,0.080823,0.501352,0.155848,0.694779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6315,"(Numero de Funcionários_Acima de 3.000, Região...","(Gênero_Masculino, Existe faculdade no Estado)",0.083964,0.402959,0.058844,0.700820,1.739182,1.0,0.025010,1.995588,0.463974,0.137460,0.498895,0.423424
6316,"(Numero de Funcionários_Acima de 3.000, Região...","(Existe faculdade no Estado, Qual sua situação...",0.070544,0.446662,0.058844,0.834146,1.867511,1.0,0.027335,3.336302,0.499785,0.128378,0.700267,0.482944
6317,"(Cor/raca/etnia_Branca, Existe faculdade no Es...","(Região onde Mora_Sudeste, Qual sua situação a...",0.079835,0.375774,0.058844,0.737069,1.961467,1.0,0.028844,2.374104,0.532706,0.148309,0.578789,0.446831
6318,"(Gênero_Masculino, Existe faculdade no Estado,...","(Região onde Mora_Sudeste, Qual sua situação a...",0.096352,0.319683,0.058844,0.610714,1.910372,1.0,0.028042,1.747602,0.527354,0.164740,0.427787,0.397392


## Pegar Regras so de região e em "se"

In [37]:
def is_regional_rule(row):
    antecedents = ','.join(row['antecedents'])  # inicia vazia
    return 'Região onde Mora_' in antecedents  # Retorna o Regiao onde mora no antecedents

rules_filtered = rules[rules.apply(is_regional_rule, axis=1)] #coloca em rules filtred so as regras com se em regiao
#manda todas as regras para a funçao pegar de volta so as regras com se Regiao onde mora
rules_filtered

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
2,(Região onde Mora_Sudeste),(Existe faculdade no Estado),0.609429,0.537164,0.497591,0.816488,1.519996,1.0,0.170228,2.522098,0.875906,0.766702,0.603505,0.871409
28,(Região onde Mora_Nordeste),(Gênero_Masculino),0.112526,0.758775,0.086029,0.764526,1.007579,1.0,0.000647,1.024423,0.008476,0.109553,0.023841,0.438952
29,(Região onde Mora_Sudeste),(Gênero_Masculino),0.609429,0.758775,0.458706,0.752682,0.991970,1.0,-0.003713,0.975364,-0.020305,0.504351,-0.025258,0.678609
31,(Região onde Mora_Sul),(Gênero_Masculino),0.200275,0.758775,0.153476,0.766323,1.009948,1.0,0.001512,1.032301,0.012316,0.190517,0.031291,0.484295
76,(Região onde Mora_Sudeste),(Cor/raca/etnia_Branca),0.609429,0.645217,0.391604,0.642575,0.995905,1.0,-0.001610,0.992608,-0.010417,0.453748,-0.007447,0.624754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6309,"(Existe faculdade no Estado, Numero de Funcion...","(Gênero_Masculino, Qual sua situação atual de ...",0.078458,0.608052,0.058844,0.750000,1.233447,1.0,0.011137,1.567791,0.205377,0.093750,0.362160,0.423387
6310,"(Existe faculdade no Estado, Numero de Funcion...","(Cor/raca/etnia_Branca, Qual sua situação atua...",0.092567,0.515485,0.058844,0.635688,1.233183,1.0,0.011127,1.329944,0.208380,0.107143,0.248088,0.374920
6315,"(Numero de Funcionários_Acima de 3.000, Região...","(Gênero_Masculino, Existe faculdade no Estado)",0.083964,0.402959,0.058844,0.700820,1.739182,1.0,0.025010,1.995588,0.463974,0.137460,0.498895,0.423424
6316,"(Numero de Funcionários_Acima de 3.000, Região...","(Existe faculdade no Estado, Qual sua situação...",0.070544,0.446662,0.058844,0.834146,1.867511,1.0,0.027335,3.336302,0.499785,0.128378,0.700267,0.482944


## Regras Boas

In [38]:
# Lift é  o programa testando se o "se" afeta positivamente ou negativamente
good_rules = rules_filtered[
    (rules_filtered['confidence'] >= 0.8) & (rules_filtered['lift'] >= 1.2) # - Lift >= 1.2 → o SE realmente aumenta a chance do ENTÃO (mais provável do que o acaso)
]
good_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
2,(Região onde Mora_Sudeste),(Existe faculdade no Estado),0.609429,0.537164,0.497591,0.816488,1.519996,1.0,0.170228,2.522098,0.875906,0.766702,0.603505,0.871409
200,"(Região onde Mora_Sudeste, Faixa idade_22-24)",(Existe faculdade no Estado),0.065726,0.537164,0.055058,0.837696,1.559478,1.0,0.019753,2.851664,0.383999,0.100503,0.649328,0.470097
211,"(Região onde Mora_Sudeste, Faixa idade_30-34)",(Existe faculdade no Estado),0.162078,0.537164,0.132829,0.819533,1.525665,1.0,0.045766,2.564653,0.411194,0.234508,0.610084,0.533405
214,"(Região onde Mora_Sudeste, Faixa idade_35-39)",(Existe faculdade no Estado),0.071920,0.537164,0.061253,0.851675,1.585501,1.0,0.022620,3.120407,0.397902,0.111809,0.679529,0.482852
218,"(Região onde Mora_Sudeste, Gênero_Feminino)",(Existe faculdade no Estado),0.146937,0.537164,0.121817,0.829040,1.543363,1.0,0.042887,2.707271,0.412706,0.216646,0.630624,0.527909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,"(Região onde Mora_Sudeste, Nível de Ensino_Pós...","(Existe faculdade no Estado, Qual sua situação...",0.058500,0.446662,0.051961,0.888235,1.988607,1.0,0.025832,4.950918,0.528025,0.114655,0.798017,0.502284
6283,"(Numero de Funcionários_Acima de 3.000, Área d...",(Existe faculdade no Estado),0.058156,0.537164,0.050241,0.863905,1.608270,1.0,0.019002,3.400835,0.401567,0.092172,0.705955,0.478718
6286,"(Região onde Mora_Sudeste, Numero de Funcionár...","(Existe faculdade no Estado, Qual sua situação...",0.062285,0.446662,0.050241,0.806630,1.805906,1.0,0.022421,2.861548,0.475903,0.109527,0.650539,0.459555
6308,"(Numero de Funcionários_Acima de 3.000, Região...",(Existe faculdade no Estado),0.066414,0.537164,0.058844,0.886010,1.649421,1.0,0.023168,4.060330,0.421736,0.108023,0.753715,0.497778


## Top 10 melhores regras

In [40]:

good_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='confidence', ascending=False).head(10)
#pega das colunas as regras com maior precição (confidence) e lista ela de forma decrescente (maior ao menor)
#depois pega somenta as 10 primeiras com .head(10)
good_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
2,(Região onde Mora_Sudeste),(Existe faculdade no Estado),0.609429,0.537164,0.497591,0.816488,1.519996,1.0,0.170228,2.522098,0.875906,0.766702,0.603505,0.871409
200,"(Região onde Mora_Sudeste, Faixa idade_22-24)",(Existe faculdade no Estado),0.065726,0.537164,0.055058,0.837696,1.559478,1.0,0.019753,2.851664,0.383999,0.100503,0.649328,0.470097
211,"(Região onde Mora_Sudeste, Faixa idade_30-34)",(Existe faculdade no Estado),0.162078,0.537164,0.132829,0.819533,1.525665,1.0,0.045766,2.564653,0.411194,0.234508,0.610084,0.533405
214,"(Região onde Mora_Sudeste, Faixa idade_35-39)",(Existe faculdade no Estado),0.071920,0.537164,0.061253,0.851675,1.585501,1.0,0.022620,3.120407,0.397902,0.111809,0.679529,0.482852
218,"(Região onde Mora_Sudeste, Gênero_Feminino)",(Existe faculdade no Estado),0.146937,0.537164,0.121817,0.829040,1.543363,1.0,0.042887,2.707271,0.412706,0.216646,0.630624,0.527909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,"(Região onde Mora_Sudeste, Nível de Ensino_Pós...","(Existe faculdade no Estado, Qual sua situação...",0.058500,0.446662,0.051961,0.888235,1.988607,1.0,0.025832,4.950918,0.528025,0.114655,0.798017,0.502284
6283,"(Numero de Funcionários_Acima de 3.000, Área d...",(Existe faculdade no Estado),0.058156,0.537164,0.050241,0.863905,1.608270,1.0,0.019002,3.400835,0.401567,0.092172,0.705955,0.478718
6286,"(Região onde Mora_Sudeste, Numero de Funcionár...","(Existe faculdade no Estado, Qual sua situação...",0.062285,0.446662,0.050241,0.806630,1.805906,1.0,0.022421,2.861548,0.475903,0.109527,0.650539,0.459555
6308,"(Numero de Funcionários_Acima de 3.000, Região...",(Existe faculdade no Estado),0.066414,0.537164,0.058844,0.886010,1.649421,1.0,0.023168,4.060330,0.421736,0.108023,0.753715,0.497778


## Quais Regras aparecem mais em quais regiões

In [49]:
# Dicionário para guardar contagens por região
regiao_consequentes = defaultdict(list) #dicionario em que o nome da regiao é a chave que guarda os atributos

# Para cada regra, associa a região (do SE) com o consequente (ENTÃO)
for _, row in rules_filtered.iterrows(): #Percorre todas as regras filtradas com Região onde Mora no "se"
    for regiao in row['antecedents']: #olhar os itens da condição "se" da regra
        if 'Região onde Mora_' in regiao: #Verifica se "se " uma região. Ex: n pegar se Junior então Pleno
            for resultado in row['consequents']: #passa no então da regra
                regiao_consequentes[regiao].append(resultado) #coloca a regra se passar em todos os testes

# Contar os consequentes mais frequentes por região
#regiao = nome da regiao. Ex: Sudeste
# consequents é a lista de atributos que apareceram no resultado das regras daquela região
for regiao, consequents in regiao_consequentes.items(): # Passar por cada região do dicionário regiao_consequentes
    print(f"\n• Região: {regiao.replace('Região onde Mora_', '')}") #Imprimir o nome da região
    mais_comuns = Counter(consequents).most_common(5) #Counter conta numero de vezes que cada atributo apareceu na lista de resultados na regiao
    for atributo, vezes in mais_comuns: #imprime todos os atributos mais comuns
        print(f"  - {atributo}: apareceu em {vezes} regras") #imrpime nome do atributo e o numero de vezes que apareceu nas regras


• Região: Sudeste
  - Qual sua situação atual de trabalho?_Empregado (CLT): apareceu em 806 regras
  - Existe faculdade no Estado: apareceu em 733 regras
  - Gênero_Masculino: apareceu em 623 regras
  - Cor/raca/etnia_Branca: apareceu em 293 regras
  - Numero de Funcionários_Acima de 3.000: apareceu em 22 regras

• Região: Nordeste
  - Gênero_Masculino: apareceu em 2 regras
  - Qual sua situação atual de trabalho?_Empregado (CLT): apareceu em 2 regras

• Região: Sul
  - Qual sua situação atual de trabalho?_Empregado (CLT): apareceu em 33 regras
  - Cor/raca/etnia_Branca: apareceu em 31 regras
  - Gênero_Masculino: apareceu em 26 regras
